# TensorflowとKerasを用いて多層パーセプトロンでディープラーニング
tensorflowとkerasを用いてmnistという手書き文字のデータを学習させる。多層パーセプトロンから成るニューラルネットをkerasで組み、学習させて正確性を検証する。

## 1. 必要なモジュールのimport
      
Pythonで書かれた定義や文が入ったファイルのことをモジュールと呼び、importで呼び出しておくことでその名前を書くだけで中身のプログラムを動作させることができるようになる。   
    
この時点では名前を使えるように設定するだけなので、何らかのプログラムが実行されるわけではない。   
      
import AAAA as A とすれば、長い名前のファイルを簡単な名前に変えて呼び出すことができる。    
      
深い階層にあるファイルでも、from AAAA.BBBB import CCCCと書くことで呼び出すことができるようになる。    
      
jupyterにあらかじめ準備されている便利な機能を使用するためには、%に続いて使用する機能を記載する。

In [ ]:
import tensorflow as tf                          # tensorflowをtfという名前に変えてimport
                                                   # 詳細：https://www.tensorflow.org
import keras                                     # tensorflowを簡単に使うためのkerasをimport
                                                   # 詳細：https://keras.io/ja/
from keras.models import Sequential              # kerasの層を積み上げていく機能であるSequentialのimport
                                                   # 詳細：https://keras.io/ja/getting-started/sequential-model-guide/
from keras.layers import Dense, Activation       # kerasの層を構成するDense, Activationの機能をimport
                                                   # 詳細：https://keras.io/ja/layers/core/
from keras.datasets import mnist                 # kerasにあらかじめ準備されているMNISTという手書き文字のデータをimport
                                                   # 詳細：https://keras.io/ja/datasets/
from keras.utils import np_utils                 # データの変形などの便利な機能を持つnp_utilsをimport
                                                   # 詳細：https://keras.io/ja/utils/np_utils/
from keras.optimizers import SGD                 # 学習を進めていく方法の一つであるSGDという機能をimport
                                                   # 詳細：https://keras.io/ja/optimizers/
import numpy as np                               # 多数の計算を迅速に進めていくnumpyをnpという名前に変えてimport
                                                   # 詳細：http://rest-term.com/archives/2999/
import matplotlib.pyplot as plt                  # matplotlib の pyplot という図を書く機能をpltという名前でimport
                                                   # 詳細：http://iatlex.com/python/matplotlib_intro1/
%matplotlib inline
                                                 # 図をjupyter内に表示するためのマジックコマンド

## 2.MNISTデータの読み込み
          
mnist.load_data()は60,000枚の28x28ピクセルの白黒画像データとその正解の数字，10,000枚のテスト用画像データとその正解の数字を出力する。    

In [ ]:
mnist.load_data()

mnist.load_data()をそのまま実行するだけだと上のように書き出されておしまいなので、それぞれ名前をつけて保存しておく。

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

60,000枚の28x28ピクセルの白黒画像データにはX_train, その正解の数字の配列にはy_trainと名前をつけた。同様に10,000枚のテスト用画像データにはX_test、その正解の数字の配列にはy_testと名前をつけた。   

それぞれ上記のX_train, X_testはshapeが(60000, 28, 28)、(10000, 28, 28)の白黒画像データの配列であり、y_train, y_testはshapeが (60000,)、(10000,)のカテゴリラベル(0-9の整数)の配列である。    
    
念のため形を確認してみる。print()で内容を書き出すことができる。

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## 3.読み込んだデータの可視化(参考)
読み込んだMNISTのデータのうち、X_testの最初の100枚の画像を下記のプログラムで描出してみる。実行に少し時間がかかるが、0〜9までの手書き文字のデータであることが確認できる。下記のプログラムの実行は今回のニューラルネットワークの学習にとっては必須ではない。

In [ ]:
fig = plt.figure(figsize=(12, 50))

for i in range(100):
    plt.subplot(20, 5, i+1)
    plt.imshow((X_test[i]).astype(np.int32).reshape(28, 28), cmap='gray')
    plt.title("No.{0}".format(i))
    plt.axis("off")
plt.tight_layout()

ためしにy_testの中身も見てみる。

In [ ]:
print(y_test)

        
## 4.読み込んだデータを多層のパーセプトロンで学習するための形に変形する
          
最も一般的に使われるニューラルネットワークである多層のパーセプトロン（MLP:multi-layer perceptrons）にデータを入力するため、読み込んだMNISTを適した形に変形する。    
    
numpyのarrayは.reshapeで形を変形できる。例えば60000枚の28x28ピクセルの画像データは、.reshape(60000,784)で784個数字が並んだベクトルが60000セットある状態に変形できる。    
     
.shape[0]は下記の場合、画像の枚数を指す。X_train.shape = (60000, 28, 28)であり、X_train.shape[0] = 60000, X_train.shape[1] = 28, X_train.shape[2] = 28である。   
     
Tensorflowは32ビット浮動小数点の数字を受け入れるため、.astype('float32')で32ビット浮動小数点の数字にする。(X_train /= 255.0はX_train = X_train/255.0と同じ意味である)    
     
X_train、X_testに含まれる数字は0-255の整数であるが、計算の便宜上255で割って全ての数字を0-1の間に圧縮する。    
     
y_train、y_testは0-9の整数の羅列であるが、ニューラルネットワークの出力と比較するため、np_utils.to_categoficalで10分類のone-hot表現に変形する。これにより、例えば4という数字は[0,0,0,0,1,0,0,0,0,0]という表現に置き換わる。

In [ ]:
X_train = X_train.reshape(X_train.shape[0],784).astype('float32')
X_test = X_test.reshape(X_test.shape[0],784).astype('float32')
X_train /= 255.0
X_test /= 255.0
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

念のため形を確認してみる。

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## 5.Kerasを用いてニューラルネットを組む
    
Kerasを用いることで、直感的にニューラルネットを組むことができる。  
    
具体的には、まずSequential()で層を重ねていく入れ物であることを明示し、.addでそこに層をinput側から重ねていく。   
    
下記のニューラルネットでは、最初のDense層は784個の入力されたデータを64個のユニットに伝達されるようにプログラムする。その際に用いる活性化関数をActivation関数で"relu"と定義する。   

Activation関数は"sigmoid"や"tanh"としても良いが、近年では"relu"が用いられることが多い。    
     
次に64個のユニットから10個のユニットに伝達されるようにプログラムする。最終のユニット数が10個である理由は、今回の問題が10種類の分類問題であるからである。    

最終層の活性化関数は、分類問題の場合には"softmax"を用いる。"softmax"では、出力された10個の数字が必ず合計で1になる。このため、"softmax"で出力された数字は確率に置き換えて考えることができる。

In [ ]:
model = Sequential()
model.add(Dense(input_dim=784, units=64, activation="relu"))
model.add(Dense(units=10, activation="softmax"))

上のプログラムで組んだニューラルネットを図示すると下図のようになる。

<p style="text-align:center;"><img src="01.png" width="400" height="400" ></p>

ちなみにKerasを使わないで素のTensorflowのみで上のニューラルネットを記述する場合、下記のようになる。

<p style="text-align:center;"><img src="tensormnist.png" width="600" height="600" ></p>

## 6.作成したニューラルネットをどのように学習させるかを設定する
上のプログラムで作成したニューラルネットを、どのように学習させるか設定する。    
    
具体的には.compile()で損失関数(loss)、最適化アルゴリズム(optimizer)、評価関数(metrics)を与えてニューラルネットを完成させる。    
    
一般的には分類問題であればlossはcategorical_crossentropyを用いる。    
    
oprimizerは下記では確率的勾配降下法（SGD, Stochastic Gradient Descent）を用いており、lrはlearning rateを示す。learning rateが大きいほど、一回の学習でのパラメータの更新は大きくなる。    

分類問題であるため、評価関数はaccuracyを用いる。

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01), metrics=["accuracy"])

## 7.学習
.fit関数を用いて学習データ、正解データ、エポック数、検証用に用いるデータの割合、バッチサイズを与えて学習を開始する。 
    
エポック数とは、データをニューラルネットワークに何周させるかという数字である。エポック数が多ければ多いほどたくさん学習させることになるが、時間がかかる。ここではepocns=10と設定している。    
    
バッチサイズは、一度に何個のデータをまとめてニューラルネットワークに入力するかというサイズである。画像データを一枚一枚入力していては時間がかかるので、通常は複数のデータをまとめて入力して重みの更新をさせる。ここではbatch_size=100と設定し、100枚の画像データをまとめて入力し、ニューラルネットワークから出力された誤差の平均を使って重みを更新していくようにしている。

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=100)

## 8.テストデータを用いて学習結果を検証する
あらかじめとっておいたテスト用データ(X_test, y_test)を用いて、今学習させたニューラルネットワークの正確性を検証する。    
.predictでテスト用データを与え、予測結果を出力する。    
    
ニューラルネットワークから出力されるデータの形は、[0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.9, 0.0, 0.0]のように10個の数字で表現される合計1のベクトルとなっている。この形から、np.argmaxで最大の数値が存在するインデックス(位置)を得る。例えば、np.argmax([0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.9, 0.0, 0.0]) = 7となる。    
    
y_pred（モデルによる予測）とy_test（正解データ）から精度を算出する。

In [ ]:
y_pred = model.predict(X_test, batch_size=100)
y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import f1_score
print('accuracy:',f1_score(Y_test, y_pred, average='macro'))

## 9.画像を用いて予測結果の検証(参考)
    
MNISTのX_testを、正解と予測結果と合わせて可視化して結果を眺める。

In [ ]:
fig = plt.figure(figsize=(12, 50))

for i in range(100):
    plt.subplot(20, 5, i+1)
    plt.imshow((X_test[i]* 255).astype(np.int32).reshape(28, 28), cmap='gray')
    if Y_test[i] == y_pred[i]:
        plt.title("No.{0} / Answer:{1}, Predict:{2}".format(i, Y_test[i], y_pred[i]))
    else:
        plt.title("No.{0} / Answer:{1}, Predict:{2}".format(i, Y_test[i], y_pred[i]), color="red")
    plt.axis("off")
plt.tight_layout()

## 10.ネットワークを変えてやってみる
kerasによるニューラルネットワーク構築部分を変えて、精度が改善しないか試行する。

In [ ]:
model = Sequential()
model.add(Dense(input_dim=784, units=400, activation="relu"))
model.add(Dense(units=200, activation="relu"))
model.add(Dense(units=10, activation="softmax"))

In [ ]:
#モデルをコンパイルして学習させてみる
model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01), metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=100)

In [ ]:
#最終的な精度を計算する
y_pred = model.predict(X_test, batch_size=100)
y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import f1_score
print('accuracy:',f1_score(Y_test, y_pred, average='macro'))